In [20]:
!pip install nltk


In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download('stopwords')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [23]:
df = pd.read_csv('All_Extracted_Data.csv')
df.head()

,product,title,content,date,author,rating,url
0,Intel Core I5 12400F 12 Gen Generation Desktop...,Very Great Processor . . .,The processor was very power efficient i devel...,"Apr, 2024",ARUN,5.0,https://www.amazon.in/Intel-Generation-Desktop...
1,Intel Core I5 12400F 12 Gen Generation Desktop...,Got it under 10k,Try to get it under 10k and check the rankings...,"May, 2024",Ayush,5.0,https://www.amazon.in/Intel-Generation-Desktop...
2,Intel Core I5 12400F 12 Gen Generation Desktop...,Highest Price to Performance processor for gam...,I upgrade from 9400f. The performance gain is ...,"Mar, 2024",Hruaia,5.0,https://www.amazon.in/Intel-Generation-Desktop...
3,Intel Core I5 12400F 12 Gen Generation Desktop...,Nice processor under 12k,Best ÃÂ°ÃÂÃÂÃÂ gaming and multitasking ...,"Apr, 2024",Mr.GeAr,5.0,https://www.amazon.in/Intel-Generation-Desktop...
4,Intel Core I5 12400F 12 Gen Generation Desktop...,12400f,"It runs well for productivity tasks, 6 cores c...","Jan, 2024",Joy Mukherjee,5.0,https://www.amazon.in/Intel-Generation-Desktop...


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3064 entries, 0 to 3063
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   product  2974 non-null   object
 1   title    3064 non-null   object
 2   content  2981 non-null   object
 3   date     2892 non-null   object
 4   author   3064 non-null   object
 5   rating   3064 non-null   object
 6   url      3064 non-null   object
dtypes: object(7)
memory usage: 167.7+ KB


In [25]:
import re
df = df.astype(str)
# Function to preprocess the text: remove stop words, punctuations, and other unnecessary characters
def clean_text(text):
    text = text.lower()  # convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    text = re.sub(r'\\xa0|\\u00|\\x|\\x9f|\\x9c|\\x8c', '', text)  # remove unicode junk
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Remove non-ASCII characters
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra spaces
    return text

# Apply the text cleaning function to title and content
df['title'] = df['title'].apply(clean_text)
df['content'] = df['content'].apply(clean_text)

# Function to categorize the product based on the model (i3, i5, i7, i9)
def extract_category(product_name):
    if 'i3' in product_name.lower():
        return 'i3'
    elif 'i5' in product_name.lower():
        return 'i5'
    elif 'i7' in product_name.lower():
        return 'i7'
    elif 'i9' in product_name.lower():
        return 'i9'
    else:
        return 'other'

# Generate a 'category' column using the updated extraction logic
df['category'] = df['product'].apply(extract_category)

# Convert 'date' to a consistent format (DD-MM-YYYY)
df['date'] = pd.to_datetime(df['date'], format='%b, %Y').dt.strftime('%d-%m-%Y')

# Remove non-numeric characters, only keep digits (this can be adjusted based on your requirements)
df['rating'] = df['rating'].str.extract('(\d+)')[0]  # Extract digits
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')  # Convert to numeric, non-convertible will become NaN

#Convert the rating from float with 1 decimal to int (zero decimal points)
df['rating'] = df['rating'].astype(float).round(0).astype(int)

# Add a serial number (sno)
df.insert(0, 'sno', range(1, 1 + len(df)))

# Drop the 'url' column
df = df.drop(columns=['url'])

df.head(10)

,sno,product,title,content,date,author,rating,category
0,1,Intel Core I5 12400F 12 Gen Generation Desktop...,very great processor,the processor was very power efficient i devel...,01-04-2024,ARUN,5,i5
1,2,Intel Core I5 12400F 12 Gen Generation Desktop...,got it under 10k,try to get it under 10k and check the rankings...,01-05-2024,Ayush,5,i5
2,3,Intel Core I5 12400F 12 Gen Generation Desktop...,highest price to performance processor for gaming,i upgrade from 9400f the performance gain is a...,01-03-2024,Hruaia,5,i5
3,4,Intel Core I5 12400F 12 Gen Generation Desktop...,nice processor under 12k,best gaming and multitasking processor at the ...,01-04-2024,Mr.GeAr,5,i5
4,5,Intel Core I5 12400F 12 Gen Generation Desktop...,12400f,it runs well for productivity tasks 6 cores cp...,01-01-2024,Joy Mukherjee,5,i5
5,6,Intel Core I5 12400F 12 Gen Generation Desktop...,just as expected,got at the cheapest price of 10250rs in the sa...,01-11-2023,Sagar Jadhav,5,i5
6,7,Intel Core I5 12400F 12 Gen Generation Desktop...,bang for the buck,this is the king in pricetoperformance ratiopa...,01-12-2023,Asad Shirazi,5,i5
7,8,Intel Core I5 12400F 12 Gen Generation Desktop...,great product from intel,i like intel processors i found this and liked...,01-11-2023,Hari,5,i5
8,9,Intel Core I5 12400F 12 Gen Generation Desktop...,i am used processor good condition,i am used processor gigabyte motherboard worki...,01-11-2023,dharm,5,i5
9,10,Intel Core I5 12400F 12 Gen Generation Desktop...,received in good condition,received in good condition good packing and go...,01-10-2023,Majid,5,i5


In [26]:
#removing stopwards for less noise and better performance

from nltk.corpus import stopwords
def remove_stop_words(text):

    # Get the set of English stop words
    stop_words = set(stopwords.words('english'))  
    
    # Remove stop words
    filtered_text = ' '.join(word for word in text.split() if word not in stop_words)
    
    return filtered_text

df['content'] = df['content'].apply(remove_stop_words)
df['title'] = df['title'].apply(remove_stop_words)

df.head(10)

,sno,product,title,content,date,author,rating,category
0,1,Intel Core I5 12400F 12 Gen Generation Desktop...,great processor,processor power efficient develop android apps...,01-04-2024,ARUN,5,i5
1,2,Intel Core I5 12400F 12 Gen Generation Desktop...,got 10k,try get 10k check rankings details sections bu...,01-05-2024,Ayush,5,i5
2,3,Intel Core I5 12400F 12 Gen Generation Desktop...,highest price performance processor gaming,upgrade 9400f performance gain huge use cooler...,01-03-2024,Hruaia,5,i5
3,4,Intel Core I5 12400F 12 Gen Generation Desktop...,nice processor 12k,best gaming multitasking processor price 12k b...,01-04-2024,Mr.GeAr,5,i5
4,5,Intel Core I5 12400F 12 Gen Generation Desktop...,12400f,runs well productivity tasks 6 cores cpu minim...,01-01-2024,Joy Mukherjee,5,i5
5,6,Intel Core I5 12400F 12 Gen Generation Desktop...,expected,got cheapest price 10250rs sale bit worried ge...,01-11-2023,Sagar Jadhav,5,i5
6,7,Intel Core I5 12400F 12 Gen Generation Desktop...,bang buck,king pricetoperformance ratiopaired 1660s game...,01-12-2023,Asad Shirazi,5,i5
7,8,Intel Core I5 12400F 12 Gen Generation Desktop...,great product intel,like intel processors found liked working like...,01-11-2023,Hari,5,i5
8,9,Intel Core I5 12400F 12 Gen Generation Desktop...,used processor good condition,used processor gigabyte motherboard working pr...,01-11-2023,dharm,5,i5
9,10,Intel Core I5 12400F 12 Gen Generation Desktop...,received good condition,received good condition good packing got 11900...,01-10-2023,Majid,5,i5


In [29]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Convert and handle errors
mean_date = df['date'].mean()

# Fill missing dates with the mean date
df['date'].fillna(mean_date, inplace=True)

In [31]:
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3064 entries, 0 to 3063
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   sno       3064 non-null   int64         
 1   product   3064 non-null   object        
 2   title     3064 non-null   object        
 3   content   3064 non-null   object        
 4   date      3064 non-null   datetime64[ns]
 5   author    3064 non-null   object        
 6   rating    3064 non-null   int32         
 7   category  3064 non-null   object        
dtypes: datetime64[ns](1), int32(1), int64(1), object(5)
memory usage: 179.7+ KB


,sno,product,title,content,date,author,rating,category
0,1,Intel Core I5 12400F 12 Gen Generation Desktop...,great processor,processor power efficient develop android apps...,2024-01-04,ARUN,5,i5
1,2,Intel Core I5 12400F 12 Gen Generation Desktop...,got 10k,try get 10k check rankings details sections bu...,2024-01-05,Ayush,5,i5
2,3,Intel Core I5 12400F 12 Gen Generation Desktop...,highest price performance processor gaming,upgrade 9400f performance gain huge use cooler...,2024-01-03,Hruaia,5,i5
3,4,Intel Core I5 12400F 12 Gen Generation Desktop...,nice processor 12k,best gaming multitasking processor price 12k b...,2024-01-04,Mr.GeAr,5,i5
4,5,Intel Core I5 12400F 12 Gen Generation Desktop...,12400f,runs well productivity tasks 6 cores cpu minim...,2024-01-01,Joy Mukherjee,5,i5
5,6,Intel Core I5 12400F 12 Gen Generation Desktop...,expected,got cheapest price 10250rs sale bit worried ge...,2023-01-11,Sagar Jadhav,5,i5
6,7,Intel Core I5 12400F 12 Gen Generation Desktop...,bang buck,king pricetoperformance ratiopaired 1660s game...,2023-01-12,Asad Shirazi,5,i5
7,8,Intel Core I5 12400F 12 Gen Generation Desktop...,great product intel,like intel processors found liked working like...,2023-01-11,Hari,5,i5
8,9,Intel Core I5 12400F 12 Gen Generation Desktop...,used processor good condition,used processor gigabyte motherboard working pr...,2023-01-11,dharm,5,i5
9,10,Intel Core I5 12400F 12 Gen Generation Desktop...,received good condition,received good condition good packing got 11900...,2023-01-10,Majid,5,i5
